# Output Parsers

모델의 출력 형식을 지정하는 유용한 방법입니다. 일반적으로 구조화된 출력에 사용됩니다.

Output parser는 언어 모델의 응답을 구조화하는 데 도움이 되는 클래스입니다. Output parser가 지원하는 두가지 기능은 다음과 같습니다.

**1. Format Instructions** : 언어 모델의 출력 형식을 지정하는 방법에 대한 지침이 포함된 문자열을 반환하는 메서드입니다. <br>
**2. Parser**: 문자열(언어 모델의 응답으로 가정)을 가져와 이를 어떤 구조(ex. csv, json 등)로 구문 분석하는 메서드입니다.

In [1]:
# read local .env file
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) 

True

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

### Comma Separated List
- `CommaSeparatedListOutputParser` 객체 생성 
- 이 객체는 Prompt에 `"Your response should be a list of comma separated values, eg: foo, bar, baz"`라는 instruction을 추가해 준다.

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [5]:
# 프롬프트 템플릿 생성
prompt = PromptTemplate(
    template="{query}의 예시 5가지를 제공하세요.\n{format_instructions}",  
    input_variables=["query"],          # 사용자 입력으로 받을 변수 목록
    partial_variables={"format_instructions": format_instructions}   # 고정된 값으로 format_instructions 사용
)

prompt = prompt.format(query="Currencies")
print(prompt)

Currencies의 예시 5가지를 제공하세요.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


LLM 에 prompt 전달

In [6]:
output = llm.invoke(prompt)
print(output.content)

USD, EUR, JPY, GBP, AUD


### Json Format   

- `StructuredOutputParser` 객체 생성 
- 이 객체는 Prompt에 

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\```json" and "\```":

\```json  
{  
	"currency": string  // answer to the user's question  
	"abbrevation": string  // Whats the abbrebation of that currency  
}   
\```  
라는 instruction을 생성해 준다.

- ResponseSchema : prompt 구성할 json의 schema 지정
    - name : json key 지정
    - description : json value 지정  

In [7]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# 응답 스키마(ResponseSchema) 정의
response_schemas = [
    # 첫 번째 응답 스키마: 통화에 대한 일반적인 질문에 대한 답변
    ResponseSchema(
        name="currency",  # 필드 이름
        description="사용자의 질문에 답하세요."  # 필드에 대한 설명
    ),
    # 두 번째 응답 스키마: 통화의 약어를 제공하는 답변
    ResponseSchema(
        name="abbrevation",  # 필드 이름
        description="그 통화의 약어는 무엇인가요?"  # 필드에 대한 설명
    )
]

In [8]:
# StructuredOutputParser를 사용하여 출력 파서를 생성
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 생성된 출력 파서 객체 확인
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='currency', description='사용자의 질문에 답하세요.', type='string'), ResponseSchema(name='abbrevation', description='그 통화의 약어는 무엇인가요?', type='string')])

In [9]:
# 출력 포맷 지침을 가져옵니다. 
# get_format_instructions() 메서드는 StructuredOutputParser가 예상하는 출력 형식을 설명하는 문자열 반환
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"currency": string  // 사용자의 질문에 답하세요.
	"abbrevation": string  // 그 통화의 약어는 무엇인가요?
}
```


In [10]:
# PromptTemplate를 사용하여 프롬프트 템플릿을 생성합니다.
prompt_template = PromptTemplate(
    template="사용자의 질문에 가능한 최선의 답변을 제공합니다.\n{format_instructions}\n{query}",  # AI 모델이 따를 지침과 입력 변수를 정의
    input_variables=["query"],       # 사용자로부터 제공될 입력 변수
    partial_variables={"format_instructions": format_instructions}  # 사전에 정의된 포맷 지침
)

prompt_template

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"currency": string  // 사용자의 질문에 답하세요.\n\t"abbrevation": string  // 그 통화의 약어는 무엇인가요?\n}\n```'}, template='사용자의 질문에 가능한 최선의 답변을 제공합니다.\n{format_instructions}\n{query}')

- 최종 완성된 prompt

In [15]:
prompt = prompt_template.format(query="미국의 통화는 무엇인가요?")
print(prompt)

사용자의 질문에 가능한 최선의 답변을 제공합니다.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"currency": string  // 사용자의 질문에 답하세요.
	"abbrevation": string  // 그 통화의 약어는 무엇인가요?
}
```
미국의 통화는 무엇인가요?


- llm에 prompt 전달

In [16]:
output = llm.invoke(prompt)
print(output.content)

```json
{
	"currency": "United States Dollar",
	"abbrevation": "USD"
}
```
